In [1]:
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
import numpy as np
import pickle
from keras.applications.inception_resnet_v2 import InceptionResNetV2
#from keras.applications.inception_resnet_v2 import InceptionResNetV2
import pandas as pd
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import SGD

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import csv
%matplotlib inline
np.random.seed(2018)
# create the base pre-teained model
base_model = InceptionResNetV2(include_top=False, weights='imagenet')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train_set = np.zeros((25000, 224, 224 ,3))
y_train_set = np.zeros((25000, 1)).astype(int)
for i in range(0, 20):
    X_train_set[1250*i: 1250*(i+1)], y_train_set[1250*i: 1250*(i+1)] = pickle.load(open("train_image_"+str(i), mode='rb'))
print("X_train_set shape:" + str(X_train_set.shape))
print("y_train_set shape:" + str(y_train_set.shape))

X_train_set shape:(25000, 224, 224, 3)
y_train_set shape:(25000, 1)


In [3]:
random_num = np.arange(25000)
permutation = np.random.permutation(random_num.shape[0])
X_train = X_train_set[permutation,:]
y_train = y_train_set[permutation,:]
print("X_train shape:" + str(X_train.shape))
print("y_train shape:" + str(y_train.shape))

X_train shape:(25000, 224, 224, 3)
y_train shape:(25000, 1)


In [4]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's Dropout
x = Dense(1024, activation = 'relu')(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(1, activation = 'sigmoid')(x)

# this is the model we will train
model = Model(input = base_model.input, output = predictions)

# forst: train only the top layer (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non_trainable)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ["accuracy"])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [5]:
# train the model on the new data for a few epochs
model.fit(x = X_train, y = y_train, epochs = 10, batch_size = 64)

# at this point, the top layer are well trained and we can start fine-tuning
# convolutional layers from inception resnet V2. we will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
    
# we choose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:629]:
    layer.trainable = False
for layer in model.layers[629:]:
    layer.trainable = True
    
# we need to recompile the model for these modifications to take effect
# we use Adam
#model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ["accuracy"])
model.compile(optimizer=SGD(lr=0.00005, momentum=0.9), loss = 'binary_crossentropy', metrics = ["accuracy"])
# we train our model again (this time fine-yuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(x = X_train, y = y_train, epochs = 10, batch_size = 64)

Epoch 1/10
25000/25000 [==============================] - 88s 4ms/step - loss: 0.1971 - acc: 0.9258
Epoch 2/10
25000/25000 [==============================] - 97s 4ms/step - loss: 0.1116 - acc: 0.9560
Epoch 3/10
25000/25000 [==============================] - 93s 4ms/step - loss: 0.0975 - acc: 0.9606
Epoch 4/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.0858 - acc: 0.9686
Epoch 5/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.0805 - acc: 0.9689
Epoch 6/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.0723 - acc: 0.9730
Epoch 7/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.0673 - acc: 0.9748
Epoch 8/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.0641 - acc: 0.9768
Epoch 9/10
25000/25000 [==============================] - 81s 3ms/step - loss: 0.0564 - acc: 0.9793
Epoch 10/10
25000/25000 [==============================] - 107s 4ms/step - loss: 0.0554 - acc: 0.980

467 conv2d_126
468 batch_normalization_126
469 activation_126
470 conv2d_127
471 batch_normalization_127
472 activation_127
473 conv2d_125
474 conv2d_128
475 batch_normalization_125
476 batch_normalization_128
477 activation_125
478 activation_128
479 block17_13_mixed
480 block17_13_conv
481 block17_13
482 block17_13_ac
483 conv2d_130
484 batch_normalization_130
485 activation_130
486 conv2d_131
487 batch_normalization_131
488 activation_131
489 conv2d_129
490 conv2d_132
491 batch_normalization_129
492 batch_normalization_132
493 activation_129
494 activation_132
495 block17_14_mixed
496 block17_14_conv
497 block17_14
498 block17_14_ac
499 conv2d_134
500 batch_normalization_134
501 activation_134
502 conv2d_135
503 batch_normalization_135
504 activation_135
505 conv2d_133
506 conv2d_136
507 batch_normalization_133
508 batch_normalization_136
509 activation_133
510 activation_136
511 block17_15_mixed
512 block17_15_conv
513 block17_15
514 block17_15_ac
515 conv2d_138
516 batch_normaliza

In [6]:
X_train, y_train = 0, 0

In [7]:
X_train, y_train = pickle.load(open("train_image_3", mode='rb'))
print("X_train shape:" + str(X_train.shape))
print("y_train shape:" + str(y_train.shape))

X_train shape:(1250, 224, 224, 3)
y_train shape:(1250, 1)


In [8]:
### START CODE HERE ### (1 line)
pred = model.evaluate(x = X_train, y = y_train, batch_size=64)
### END CODE HERE ###

1250/1250 [==============================] - 7s 6ms/step


In [9]:
print ("Loss = " + str(pred[0]))
print ("Test Accuracy = " + str(pred[1]))

Loss = 0.0446016559900716
Test Accuracy = 0.9911999992370606


In [10]:
X_test = np.zeros((12500, 224, 224 ,3))
for i in range(0, 10):
    X_test[1250*i: 1250*(i+1)] = pickle.load(open("test_image_"+str(i), mode='rb'))

In [11]:
y_pred = model.predict(X_test, batch_size = 32)

In [12]:

print(y_pred.shape)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from ipykernel import kernelapp as app
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('submission.csv', index=None)
df.head(10)

(12500, 1)
Found 12500 images belonging to 1 classes.


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.005
4,5,0.005
5,6,0.995
6,7,0.995
7,8,0.995
8,9,0.005
9,10,0.995


In [ ]:
id = np.arange(1, 12501)
y_pred = y_pred.reshape(12500,)
with open("submission.csv","w", newline='') as csvfile: 
    writer = csv.writer(csvfile)

    # first columns_name
    writer.writerow(["id","label"])
    # the wrire 
    writer.writerows(zip(id, y_pred))

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [14]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
import numpy as np  
a = np.random.rand(3,3,3)  
a

In [36]:
b = np.random.randn(3,1)
b

array([[1.43643336],
       [0.91258855],
       [1.27570009]])

In [37]:
lable = np.arange(3)  
lable

array([0, 1, 2])

In [38]:
permutation = np.random.permutation(lable.shape[0])  
permutation

array([1, 0, 2])

In [39]:
data = a[permutation,:]
data

array([[[0.18682529, 0.19334165, 0.24877612],
        [0.53181091, 0.07715978, 0.23078542],
        [0.36373742, 0.21507477, 0.84807359]],

       [[0.7966188 , 0.97013153, 0.61428206],
        [0.93849214, 0.55366679, 0.76971369],
        [0.8921467 , 0.57324996, 0.06447697]],

       [[0.91648949, 0.37267165, 0.06008406],
        [0.09536563, 0.22085714, 0.99407111],
        [0.229904  , 0.10476899, 0.58792872]]])

In [40]:
labels = b[permutation,:]
labels

array([[0.91258855],
       [1.43643336],
       [1.27570009]])